In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [88]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

import argparse
import ast
import matplotlib.pyplot as plt
import logging
import sys  
sys.path.insert(0, '../lib')
from utils import *
logging.basicConfig(level=logging.INFO)

In [186]:
%run ../lib/mple_learn_lib.ipynb
%run ../lib/report_lib.ipynb

select optimal lambda


# MIT

Original data file is `.../data/phonedata/RData`

### Data description

Data goes from 09/14/2004 to 05/05/2005. **Symmetric** adjacency matrices. 

df[,,1] is the matrix correpsonding to the first  four hour block on day 1. <br>
df[,,2] is the matrix correpsonding to the second four hour block on day 1. <br>
... <br>
data[,,7] is the matrix correpsonding to the first  four hour block on day 2, <br>


Number of students/faculties: 96. <br>
Number of days: 232. <br>


In [36]:
args = {
    'inputdir': '../../output/mit1',
    'result_identifier': 1,
}

In [57]:
%%time
H, y, experiment_dir, sim_args = prepare_data(args)

INFO:root:Loading network statistics and edge data...


{'outdir': '../../output/mit1', 'data_name': 'mit', 'form_terms': ['edges', 'mutual'], 'diss_terms': ['edges', 'mutual'], 'true_change_points': 'None'}



INFO:root:Data has dimension (t, n, p): (232, 96, 4)


CPU times: user 3.53 s, sys: 232 ms, total: 3.76 s
Wall time: 3.81 s


In [58]:
result_dir = make_directory(experiment_dir, 'results' + str(args["result_identifier"]))
args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
save_args(args, args_dir + 'args_model.txt')
theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff.png'

In [72]:
conf = {
    'lam': 10,
    'admm_alpha': 20,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
#     'true_cp': None, #True change points (for plot).
#     "cp_thr": 3, #"Threshold for estimating change points
    "verbose": 1,
}

In [73]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=conf["solver"])
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)


Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 4.147718191146851.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0010361671447753906.
[INFO] Updating u...
[INFO] max mu :  0.9252134245493031
[INFO] dual_resnorm: 2.635203
[INFO] primal_resnorm: 0.889150
[INFO] convergence: 2.635203
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 4.549518823623657.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0010411739349365234.
[INFO] Updating u...
[INFO] max mu :  0.9758639955932075
[INFO] dual_resnorm: 1.355032
[INFO] primal_resnorm: 0.251105
[INFO] convergence: 1.355032
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 6.121779918670654.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0013577938079833984.
[INFO] Updating u...
[INFO] max mu :  0.984545119

[INFO] Newton converged in: 3.4435651302337646.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.001249074935913086.
[INFO] Updating u...
[INFO] max mu :  0.9867132392923272
[INFO] dual_resnorm: 0.122277
[INFO] primal_resnorm: 0.016579
[INFO] convergence: 0.122277
-------------------------------------------------
[INFO] ADMM step #23
[INFO] Updating theta...
[INFO] Newton converged in: 3.4546971321105957.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.00115203857421875.
[INFO] Updating u...
[INFO] max mu :  0.9877530988860979
[INFO] dual_resnorm: 0.111910
[INFO] primal_resnorm: 0.015102
[INFO] convergence: 0.111910
-------------------------------------------------
[INFO] ADMM step #24
[INFO] Updating theta...
[INFO] Newton converged in: 3.289989948272705.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.001318216323852539.
[INFO] Updating u...
[INFO] max mu :  0.9886072617299112
[INFO] dual_resnorm: 0.102994
[INFO] primal_resnorm: 0.013394
[INFO

[INFO] Newton converged in: 5.257427930831909.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0010118484497070312.
[INFO] Updating u...
[INFO] max mu :  0.9977597936240123
[INFO] dual_resnorm: 0.079790
[INFO] primal_resnorm: 0.010113
[INFO] convergence: 0.079790
-------------------------------------------------
[INFO] ADMM step #45
[INFO] Updating theta...
[INFO] Newton converged in: 5.275492191314697.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0013730525970458984.
[INFO] Updating u...
[INFO] max mu :  0.9979182067468235
[INFO] dual_resnorm: 0.073866
[INFO] primal_resnorm: 0.008975
[INFO] convergence: 0.073866
-------------------------------------------------
[INFO] ADMM step #46
[INFO] Updating theta...
[INFO] Newton converged in: 5.0266501903533936.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.001322031021118164.
[INFO] Updating u...
[INFO] max mu :  0.9980553072133183
[INFO] dual_resnorm: 0.068714
[INFO] primal_resnorm: 0.007911
[IN

[INFO] Newton converged in: 10.943627119064331.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0026941299438476562.
[INFO] Updating u...
[INFO] max mu :  0.9997476178639986
[INFO] dual_resnorm: 0.058014
[INFO] primal_resnorm: 0.011897
[INFO] convergence: 0.058014
-------------------------------------------------
[INFO] ADMM step #67
[INFO] Updating theta...
[INFO] Newton converged in: 10.761460304260254.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0023810863494873047.
[INFO] Updating u...
[INFO] max mu :  0.9997630743609197
[INFO] dual_resnorm: 0.054490
[INFO] primal_resnorm: 0.009912
[INFO] convergence: 0.054490
-------------------------------------------------
[INFO] ADMM step #68
[INFO] Updating theta...
[INFO] Newton converged in: 10.542163848876953.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.002196788787841797.
[INFO] Updating u...
[INFO] max mu :  0.9997769606963047
[INFO] dual_resnorm: 0.051398
[INFO] primal_resnorm: 0.008486
[

[INFO] Newton converged in: 15.95095682144165.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.008341073989868164.
[INFO] Updating u...
[INFO] max mu :  0.9999524843535714
[INFO] dual_resnorm: 0.036809
[INFO] primal_resnorm: 0.007031
[INFO] convergence: 0.036809
-------------------------------------------------
[INFO] ADMM step #89
[INFO] Updating theta...
[INFO] Newton converged in: 15.94457483291626.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.00793313980102539.
[INFO] Updating u...
[INFO] max mu :  0.9999544025464453
[INFO] dual_resnorm: 0.035353
[INFO] primal_resnorm: 0.006343
[INFO] convergence: 0.035353
-------------------------------------------------
[INFO] ADMM step #90
[INFO] Updating theta...
[INFO] Newton converged in: 16.101102113723755.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0077970027923583984.
[INFO] Updating u...
[INFO] max mu :  0.9999561877777532
[INFO] dual_resnorm: 0.034007
[INFO] primal_resnorm: 0.005749
[INFO

In [5]:
%run ../lib/report_lib.ipynb

In [6]:
plot_mit_cpts(
    theta, 
    output_dir=theta_plot_dir,
    exp_name="MIT",
    thr=.1,
    change_pts=None
)

NameError: name 'theta_plot_dir' is not defined

# SBM

### sbm1

In [193]:
args = {
    'inputdir': "../../output/sbm1",
    "result_identifier": 1,
}

H, y, experiment_dir, sim_args = prepare_data(args)
result_dir = make_directory(experiment_dir, 'results' + str(args["result_identifier"]))
args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
save_args(args, args_dir + 'args_model.txt')
theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff_' + sim_args["data_name"][:-4] + '.png'

INFO:root:Loading network statistics and edge data...


{"outdir": "../../output/sbm1", "data_name": "sbm_rho00n100t500.rds", "form_terms": ["edges", "mutual"], "diss_terms": ["edges", "mutual"], "true_change_points": [167, 333]}



INFO:root:Data has dimension (t, n, p): (500, 100, 4)


In [205]:
conf = {
    'lam': 30,
    'admm_alpha': 50,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
#     'true_cp': None, #True change points (for plot).
#     "cp_thr": 3, #"Threshold for estimating change points
    "verbose": 1,
}

In [207]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=conf["solver"])
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 42.15334606170654.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005662441253662109.
[INFO] Updating u...
[INFO] max mu :  0.9645473157252707
[INFO] dual_resnorm: 0.486801
[INFO] primal_resnorm: 1.974075
[INFO] convergence: 1.974075
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 19.477690935134888.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005860328674316406.
[INFO] Updating u...
[INFO] max mu :  0.9435205814328623
[INFO] dual_resnorm: 0.177005
[INFO] primal_resnorm: 0.184904
[INFO] convergence: 0.184904
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 23.17600989341736.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005581378936767578.
[INFO] Updating u...
[INFO] max mu :  0.91743226

[INFO] Newton converged in: 12.926882028579712.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0007481575012207031.
[INFO] Updating u...
[INFO] max mu :  0.5
[INFO] dual_resnorm: 0.000581
[INFO] primal_resnorm: 0.000282
[INFO] convergence: 0.000581
-------------------------------------------------
[INFO] ADMM step #21
[INFO] Updating theta...
[INFO] Newton converged in: 11.915763139724731.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0006430149078369141.
[INFO] Updating u...
[INFO] max mu :  0.5
[INFO] dual_resnorm: 0.000332
[INFO] primal_resnorm: 0.000244
[INFO] convergence: 0.000332
-------------------------------------------------
[INFO] ADMM step #22
[INFO] Updating theta...
[INFO] Newton converged in: 11.80446481704712.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005078315734863281.
[INFO] Updating u...
[INFO] max mu :  0.5
[INFO] dual_resnorm: 0.000209
[INFO] primal_resnorm: 0.000214
[INFO] convergence: 0.000214
-----------------

In [203]:
plot_theta(
    theta, 
    output_dir=theta_plot_dir,
    fig_title="",
    thr=6e-6,
    change_pts=sim_args['true_change_points']
)

The estimated change points are [  1   2  82 108 137 138 167 168 180 227 237 265 333 334 346 488]


In [5]:
# theta = np.loadtxt(fname="../../output/sbm1/results1/theta_sbm_rho00n100t500.rds.txt")
# y = np.loadtxt(fname="../../output/sbm1/sbm_rho00n100t500.rds_y.txt")

### SBM2

In [182]:
args = {
    'inputdir': "../../output/sbm2",
    "result_identifier": 1,
}

H, y, experiment_dir, sim_args = prepare_data(args)

result_dir = make_directory(experiment_dir, 'results' + str(args["result_identifier"]))
args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
save_args(args, args_dir + 'args_model.txt')
theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff_' + sim_args["data_name"][:-4] + '.png'

INFO:root:Loading network statistics and edge data...


{"outdir": "../../output/sbm2", "data_name": "sbm_rho05n100t500.rds", "form_terms": ["edges", "mutual"], "diss_terms": ["edges", "mutual"], "true_change_points": [167, 333]}



INFO:root:Data has dimension (t, n, p): (500, 100, 4)


In [183]:
conf = {
    'lam': 20,
    'admm_alpha': 100,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
#     'true_cp': None, #True change points (for plot).
#     "cp_thr": 3, #"Threshold for estimating change points
    "verbose": 1,
}

In [185]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=conf["solver"])
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 15.130270004272461.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0004761219024658203.
[INFO] Updating u...
[INFO] max mu :  0.9362073952126797
[INFO] dual_resnorm: 0.846960
[INFO] primal_resnorm: 0.135621
[INFO] convergence: 0.846960
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 13.43637204170227.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0014998912811279297.
[INFO] Updating u...
[INFO] max mu :  0.9377083789009178
[INFO] dual_resnorm: 0.126505
[INFO] primal_resnorm: 0.102043
[INFO] convergence: 0.126505
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 14.054702043533325.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0007159709930419922.
[INFO] Updating u...
[INFO] max mu :  0.9044442

[INFO] Newton converged in: 10.676789999008179.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005960464477539062.
[INFO] Updating u...
[INFO] max mu :  0.6968341104346768
[INFO] dual_resnorm: 0.000373
[INFO] primal_resnorm: 0.000146
[INFO] convergence: 0.000373
-------------------------------------------------
[INFO] ADMM step #21
[INFO] Updating theta...
[INFO] Newton converged in: 10.617364883422852.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005998611450195312.
[INFO] Updating u...
[INFO] max mu :  0.696839139916935
[INFO] dual_resnorm: 0.000139
[INFO] primal_resnorm: 0.000133
[INFO] convergence: 0.000139
-------------------------------------------------
[INFO] ADMM step #22
[INFO] Updating theta...
[INFO] Newton converged in: 9.831367015838623.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005321502685546875.
[INFO] Updating u...
[INFO] max mu :  0.6968375074627436
[INFO] dual_resnorm: 0.000051
[INFO] primal_resnorm: 0.000119
[I

In [187]:
plot_theta(
    theta, 
    output_dir=theta_plot_dir,
    fig_title="",
    thr=6e-6,
    change_pts=sim_args['true_change_points'],
)

The estimated change points are [167 168 333 334]


### SBM 3

In [189]:
args = {
    'inputdir': "../../output/sbm3",
    "result_identifier": 1,
}

H, y, experiment_dir, sim_args = prepare_data(args)

result_dir = make_directory(experiment_dir, 'results' + str(args["result_identifier"]))
args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
save_args(args, args_dir + 'args_model.txt')
theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff_' + sim_args["data_name"][:-4] + '.png'

INFO:root:Loading network statistics and edge data...


{"outdir": "../../output/sbm3", "data_name": "sbm_rho09n100t500.rds", "form_terms": ["edges", "mutual"], "diss_terms": ["edges", "mutual"], "true_change_points": [167, 333]}



INFO:root:Data has dimension (t, n, p): (500, 100, 4)


In [190]:
conf = {
    'lam': 20,
    'admm_alpha': 100,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
    "verbose": 1,
}

In [191]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=conf["solver"])
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 16.75276803970337.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0006949901580810547.
[INFO] Updating u...
-------------------------------------------------
[INFO] decreasing alpha to 50.0
[INFO] max mu :  0.951598717258875
[INFO] dual_resnorm: 1.586914
[INFO] primal_resnorm: 0.152432
[INFO] convergence: 1.586914
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 18.494092226028442.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.000985860824584961.
[INFO] Updating u...
[INFO] max mu :  0.9738148399603936
[INFO] dual_resnorm: 0.462094
[INFO] primal_resnorm: 0.158194
[INFO] convergence: 0.462094
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 18.315818071365356.
[INFO] Updating z...
[INFO] Group fused Lasso conv

[INFO] Newton converged in: 8.78452205657959.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.00032520294189453125.
[INFO] Updating u...
-------------------------------------------------
[INFO] decreasing alpha to 25.0
[INFO] max mu :  0.9372968628486523
[INFO] dual_resnorm: 0.003396
[INFO] primal_resnorm: 0.000252
[INFO] convergence: 0.003396
-------------------------------------------------
[INFO] ADMM step #21
[INFO] Updating theta...
[INFO] Newton converged in: 10.53165888786316.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005080699920654297.
[INFO] Updating u...
[INFO] max mu :  0.9373925503921943
[INFO] dual_resnorm: 0.002612
[INFO] primal_resnorm: 0.000350
[INFO] convergence: 0.002612
-------------------------------------------------
[INFO] ADMM step #22
[INFO] Updating theta...
[INFO] Newton converged in: 10.490019083023071.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0004489421844482422.
[INFO] Updating u...
[INFO] max mu :  0

In [192]:
plot_theta(
    theta, 
    output_dir=theta_plot_dir,
    fig_title="",
    thr=6e-6,
    change_pts=sim_args['true_change_points'],
)

The estimated change points are [167 168 333 334]


### SBM 4

In [208]:
args = {
    'inputdir': "../../output/sbm4",
    "result_identifier": 1,
}

H, y, experiment_dir, sim_args = prepare_data(args)

result_dir = make_directory(experiment_dir, 'results' + str(args["result_identifier"]))
args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
save_args(args, args_dir + 'args_model.txt')
theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff_' + sim_args["data_name"][:-4] + '.png'

INFO:root:Loading network statistics and edge data...


{"outdir": "../../output/sbm4", "data_name": "sbm_rho09n200t500.rds", "form_terms": ["edges", "mutual"], "diss_terms": ["edges", "mutual"], "true_change_points": [167, 333]}



INFO:root:Data has dimension (t, n, p): (500, 200, 4)


In [209]:
conf = {
    'lam': 20,
    'admm_alpha': 100,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'newton_max_steps': 100,
    'converge_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
    "verbose": 1,
}

In [210]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=conf["solver"])
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 187.77582430839539.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0038421154022216797.
[INFO] Updating u...
-------------------------------------------------
[INFO] decreasing alpha to 50.0
[INFO] max mu :  0.9847986663493044
[INFO] dual_resnorm: 1.928008
[INFO] primal_resnorm: 0.156173
[INFO] convergence: 1.928008
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 99.93031406402588.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0016629695892333984.
[INFO] Updating u...
[INFO] max mu :  0.9890059050090101
[INFO] dual_resnorm: 0.278293
[INFO] primal_resnorm: 0.155591
[INFO] convergence: 0.278293
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 64.96576428413391.
[INFO] Updating z...
[INFO] Group fused Lasso con

[INFO] Newton converged in: 42.986490964889526.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005519390106201172.
[INFO] Updating u...
[INFO] max mu :  0.9379530107653187
[INFO] dual_resnorm: 0.003820
[INFO] primal_resnorm: 0.000567
[INFO] convergence: 0.003820
-------------------------------------------------
[INFO] ADMM step #20
[INFO] Updating theta...
[INFO] Newton converged in: 39.37859582901001.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0004012584686279297.
[INFO] Updating u...
[INFO] max mu :  0.9379940897077396
[INFO] dual_resnorm: 0.001475
[INFO] primal_resnorm: 0.000526
[INFO] convergence: 0.001475
-------------------------------------------------
[INFO] ADMM step #21
[INFO] Updating theta...
[INFO] Newton converged in: 35.49988007545471.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.000293731689453125.
[INFO] Updating u...
[INFO] max mu :  0.9379799192611148
[INFO] dual_resnorm: 0.000561
[INFO] primal_resnorm: 0.000449
[IN

In [211]:
plot_theta(
    theta, 
    output_dir=theta_plot_dir,
    fig_title="",
    thr=6e-6,
    change_pts=sim_args['true_change_points'],
)

The estimated change points are [167 168 333 334]


### SBM 5

In [212]:
args = {
    'inputdir': "../../output/sbm5",
    "result_identifier": 1,
}

H, y, experiment_dir, sim_args = prepare_data(args)

result_dir = make_directory(experiment_dir, 'results' + str(args["result_identifier"]))
args_dir = make_directory(result_dir, 'args' + str(args["result_identifier"]))
save_args(args, args_dir + 'args_model.txt')
theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff_' + sim_args["data_name"][:-4] + '.png'

INFO:root:Loading network statistics and edge data...


{"outdir": "../../output/sbm5", "data_name": "sbm_rho05n200t500.rds", "form_terms": ["edges", "mutual"], "diss_terms": ["edges", "mutual"], "true_change_points": [167, 333]}



INFO:root:Data has dimension (t, n, p): (500, 200, 4)


In [213]:
%%time
model = STERGMGraph(
    **conf,
)

model.load_H_y(H, y)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=conf["solver"])
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

Run mple to estimate theta...
[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 165.86991214752197.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.001192331314086914.
[INFO] Updating u...
[INFO] max mu :  0.9821992004511055
[INFO] dual_resnorm: 0.686583
[INFO] primal_resnorm: 2.913748
[INFO] convergence: 2.913748
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 63.882988929748535.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005970001220703125.
[INFO] Updating u...
[INFO] max mu :  0.9761050761878838
[INFO] dual_resnorm: 0.373210
[INFO] primal_resnorm: 0.211381
[INFO] convergence: 0.373210
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 73.11017394065857.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0003349781036376953.
[INFO] Updating u...
---------------------------

[INFO] Newton converged in: 39.92398524284363.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0004980564117431641.
[INFO] Updating u...
[INFO] max mu :  0.6970223826076566
[INFO] dual_resnorm: 0.000089
[INFO] primal_resnorm: 0.000208
[INFO] convergence: 0.000208
-------------------------------------------------
[INFO] ADMM step #21
[INFO] Updating theta...
[INFO] Newton converged in: 37.84638690948486.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0004971027374267578.
[INFO] Updating u...
[INFO] max mu :  0.6970137451945505
[INFO] dual_resnorm: 0.000031
[INFO] primal_resnorm: 0.000188
[INFO] convergence: 0.000188
-------------------------------------------------
[INFO] ADMM step #22
[INFO] Updating theta...
[INFO] Newton converged in: 38.03757190704346.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.0005781650543212891.
[INFO] Updating u...
-------------------------------------------------
[INFO] increasing alpha to 800.0
[INFO] max mu :  0

In [214]:
plot_theta(
    theta, 
    output_dir=theta_plot_dir,
    fig_title="",
    thr=6e-6,
    change_pts=sim_args['true_change_points'],
)

The estimated change points are [167 168 333 334]


In [234]:
# theta = np.loadtxt(fname="../../output/sbm1/results1/theta_sbm_rho00n100t500.rds.txt")
# y = np.loadtxt(fname="../../output/sbm1/sbm_rho00n100t500.rds_y.txt")
# H = np.loadtxt(fname="../../output/sbm2/sbm_rho05n100t500.rds_H.txt")

## TODO: convert list of networks to network series and compute likelihood

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
from rpy2.robjects import numpy2ri
numpy2ri.activate()


network = importr(
    'network',
    on_conflict="warn",
    robject_translations={
        'as.tibble.network': 'as_tibble_network',
        'as_tibble.network': 'as_tibble_network_'
    })

ergm = importr('ergm')
tergm = importr('tergm')
readRDS = robjects.r['readRDS']
lapply = robjects.r['lapply']

# robjects.r("options(error=recover)")

In [26]:
data = readRDS("../../data/test_nw_list.rds")
test = lapply(data, network.network)
abns = tergm.NetSeries(test)
fml_form = Formula('nwf ~ Form(~edges+mutual) + Diss(~edges+mutual)')
fml_form.environment['nwf'] = abns

# robjects.r['options'](warn=-1, message=-1)
# robjects.r("options(error=recovery)")

warn,[RTYPES.INTSXP]
message,[RTYPES.NILSXP]


In [28]:
import numpy as np
res = ergm.ergm_bridge_dindstart_llk(
    fml_form,
    coef=np.array([-2,-1,1,1]),
    verbose=False,
)

R[write to console]: Fitting the dyad-independent submodel...

R[write to console]: Bridging between the dyad-independent submodel and the full model...

R[write to console]: Setting up bridge sampling...

R[write to console]: Using 16 bridges: 
R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 5 
R[write to console]: 6 
R[write to console]: 7 
R[write to console]: 8 
R[write to console]: 9 
R[write to console]: 10 
R[write to console]: 11 
R[write to console]: 12 
R[write to console]: 13 
R[write to console]: 14 
R[write to console]: 15 
R[write to console]: 16 
R[write to console]: .

R[write to console]: Bridging finished.



In [29]:
res

array([-125.12430792])